In [1]:
dir = "/kaggle/input/training-data/training_data2.jsonl"

In [ ]:
%pip install -U -q 'google-generativeai>=0.8.3'

In [ ]:
import google.generativeai as genai


In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
from datasets import load_dataset

df = load_dataset(path = "/kaggle/input/training-data/",data_files = "training_data2.jsonl", split='train')

In [ ]:
df = df.to_pandas()
df.head(5)

In [ ]:
%pip install -U -q langchain-community

In [ ]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

In [ ]:
df_loader = DataFrameLoader(df, page_content_column= "completion")

In [ ]:
df_document = df_loader.load()
display(df_document[:2])

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1250,
                                      separator="\n",
                                      chunk_overlap=100)
texts = text_splitter.split_documents(df_document)

In [ ]:
first_doc = texts[1]
print(first_doc.page_content)

In [ ]:
from getpass import getpass
OPENAI_API_KEY = getpass("OpenAI API Key: ")

In [ ]:
pip install langchain-openai

In [ ]:

from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)
  

In [ ]:
pip install chromadb

In [ ]:
directory_cdb = '/kaggle/working/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm=OpenAI(openai_api_key=OPENAI_API_KEY,
           temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

In [ ]:
qa.invoke("What is the recommendation for amitryptilline for someone with a *17/*17 diplotype?")

In [ ]:
from langchain.agents import Tool

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Genomic Counseling tool',
        func=qa.invoke,
        description=(
            """use this tool when answering how to dose a medication given a phenotype and or a diplotype"""
        )
    )
]

In [ ]:
from langchain.agents import create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt,
)

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               handle_parsing_errors=True
                               )

In [ ]:
response = agent_executor.invoke({"input": """I have a patient that is an intermediate metabolizer for citalopram, what is the recommendation?"""})

In [ ]:
response['output']

In [ ]:
pip install streamlit

In [ ]:

pip install widgetsnbextension 


In [ ]:
pip install ipywidgets 


In [ ]:
pip install voila

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
grand = widgets.ToggleButtons(
            options=['grandson', 'granddaughter']
        )

In [ ]:
name = widgets.Text(placeholder='Your name here')

In [ ]:
button_send = widgets.Button(
                description='Send to grandma',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        print("Sent message: ")
        print(f"Dear Grandma! This is your favourite {grand.value}, {name.value}.")
        print(f"I would love to come over on {date.value} for dinner, if that's okay for you!")
        print(f"Also, if you don't mind, I'll bring along {friends.value} hungry ghosts for your delicious food!")

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [ ]:
voila notebook_name.ipynb